In [1]:
import json
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import Dataset
from transformers import (AutoTokenizer,
                          AutoModelForCausalLM,
                          BitsAndBytesConfig,
                          TrainingArguments,
                          Trainer,
                          pipeline,
                          DataCollatorForLanguageModeling,
                          PreTrainedTokenizer)
from peft import LoraConfig, get_peft_model

import huggingface_hub
import os
import logging
from tqdm import tqdm
from sklearn.metrics import (accuracy_score,
                             classification_report,
                             confusion_matrix)
from typing import List
import wandb

In [ ]:
!pip install transformers
!pip install peft
!pip install 'accelerate>=0.26.0'
!pip install -U bitsandbytes
!pip install huggingface-hub
!pip install datasets
!pip install wandb

In [2]:
cache_dir = "/home/ec2-user/SageMaker"
os.environ['HF_HOME'] = cache_dir
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

In [3]:
class CustomTextDataset(Dataset):
    """
    Custom dataset class for QEvasion Dataset.

    Attributes:
        texts (List[str]): A list of text samples to be tokenized.
        tokenizer (PreTrainedTokenizer): A tokenizer from the Hugging Face Transformers library.
    """

    def __init__(self,
                 texts: List[str],
                 tokenizer: PreTrainedTokenizer) -> None:
        self.texts = texts
        self.tokenizer = tokenizer

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = str(self.texts[idx])

        encoding = self.tokenizer(
            text,
            return_tensors='pt'
        )

        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
        }


def create_prompted_text(dataset: pd.DataFrame,
                         label_name: str) -> List[str]:
    """
    Creates prompted text for classification from the dataset.

    Args:
        dataset (pd.DataFrame): The dataset containing interview questions and 
        answers.
        label_name (str): The name of the label column for classification.
        
    Returns:
        List[str]: A list of formatted prompt texts for each interview response.
    """
    texts = []
    global class_names
    class_names = list(dataset[label_name].unique())
    class_names_text = ', '.join(class_names)

    for _, row in dataset.iterrows():
        texts.append(
            f"You will be given a part of an interview. "
            f"Classify the response to the selected question "
            f"into one of the following categories: {class_names_text}"
            f". \n\n ### Part of the interview ### \nIntervier:"
            f" {row['interview_question']} \nResponse:"
            f" {row['interview_answer']} \n\n### Selected Question ###\n"
            f"{row['question']} \n\nLabel: {row[label_name]}"
        )
    return texts


def load_qevasion_dataset(tokenizer: PreTrainedTokenizer,
                          label_name: str = "clarity_label") -> tuple:
    """
    Loads the QEvasion dataset, splits it into training and validation sets,
    and creates prompted texts for both sets.

    Args:
        tokenizer: The tokenizer to be used for text encoding.
        label_name (str): The name of the label column for classification.
        
    Returns:
        tuple: A tuple containing the training and validation datasets.
    """

    # Get train set data
    df = pd.read_csv('preprocessed_data/train_set.csv')[['question',
                                                         'interview_question',
                                                         'interview_answer',
                                                         label_name]]

    # Split train set to train and validation data
    np.random.seed(2024)
    msk = np.random.rand(len(df)) < 0.9
    train = df[msk]
    validation = df[~msk]

    train.reset_index(drop=True, inplace=True)
    validation.reset_index(drop=True, inplace=True)

    train_texts = create_prompted_text(train, label_name)
    validation_texts = create_prompted_text(validation,
                                            label_name)

    # print("Example of train test:" + train_texts[1])
    # print("Example of validation test:" + validation_texts[1])

    train_texts = train_texts[:20]
    validation_texts = validation_texts[:2]
    return (CustomTextDataset(train_texts, tokenizer),
            CustomTextDataset(validation_texts, tokenizer))


# def compute_metrics(pred):
#     references = pred.label_ids
#     generated_ids = pred.predictions.argmax(axis=-1)  # Get the predicted token IDs
#     generated_texts = [tokenizer.decode(g, skip_special_tokens=True) for g in generated_ids]  # Decode to text
#     print("----")
#     print("References:", references)
#     print("Generated Texts:", generated_texts)
#     print("----")


class CastOutputToFloat(nn.Sequential):
    def forward(self, x): return super().forward(x).to(torch.float32)


def print_trainable_parameters(model: nn.Module) -> None:
    """
    Prints the number of trainable parameters in the model.

    Args:
        model (nn.Module): The model for which to count trainable parameters.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"""trainable params: {trainable_params} || all params: {all_param}
        || trainable%: {100 * trainable_params / all_param}"""
    )


def finetuning(model_name: str,
               output_model_dir: str,
               label_taxonomy: str,
               lr: float,
               epochs: int) -> tuple:
    """
    Fine-tunes a pre-trained language model with LoRA.

    Args:
        model_name (str): The name of the pre-trained model.
        output_model_dir (str): Directory to save the fine-tuned model.
        label_taxonomy (str): The label taxonomy for the dataset.
        lr (float): Learning rate for training.
        epochs (int): Number of training epochs.

    Returns:
        tuple: The fine-tuned model and tokenizer.
    """

    model = AutoModelForCausalLM.from_pretrained(
        model_name,
        quantization_config=BitsAndBytesConfig(
            load_in_4bit=True,
            bnb_4bit_compute_dtype=torch.bfloat16
        ),
        device_map='auto',
        torch_dtype=torch.float16,
        cache_dir=cache_dir
    )

    tokenizer = AutoTokenizer.from_pretrained(model_name, cache_dir=cache_dir,
                                              trust_remote_code=True,)

    # NEW
    tokenizer.pad_token = tokenizer.eos_token

    # !TODO: Check if this helps
    # tokenizer.pad_token_id = tokenizer.eos_token_id

    for param in model.parameters():
        param.requires_grad = False  # Freeze the model - train adapters later
        if param.ndim == 1:
            # Cast the small parameters to fp32 for stability
            param.data = param.data.to(torch.float32)

    # Reduce number of stored activation
    model.gradient_checkpointing_enable()
    model.enable_input_require_grads()
    model.lm_head = CastOutputToFloat(model.lm_head)

    config = LoraConfig(
        r=16,  # Attention heads
        lora_alpha=32,  # Alpha scaling
        # target_modules=["q_proj", "v_proj"],
        lora_dropout=0.05,
        bias="none",
        task_type="CAUSAL_LM"
    )

    model = get_peft_model(model, config)
    print_trainable_parameters(model)

    # Load data
    train_data, validation_data = load_qevasion_dataset(tokenizer,
                                                        label_taxonomy)

    print(f"""Found {len(train_data)} instances for training and
    {len(validation_data) } instances for validation.""")

    grad_accum_steps = 8

    # Train model
    print("Training...")

    trainer = Trainer(
        model=model,
        train_dataset=train_data,
        eval_dataset=validation_data,
        args=TrainingArguments(
            per_device_train_batch_size=1,
            per_device_eval_batch_size=1,
            gradient_accumulation_steps=grad_accum_steps,
            eval_accumulation_steps=1,
            warmup_steps=100,
            max_steps=int((len(train_data)*epochs)/grad_accum_steps),
            learning_rate=lr,
            fp16=True,
            logging_steps=1,
            # eval_steps * int((len(train_data)*epochs)/grad_accum_steps)
            # if eval_steps < 1
            eval_steps=0.33 / epochs,
            eval_strategy="steps",
            do_eval=True,
            report_to="wandb",
            # save_steps= 2,
            # num_train_epochs=epochs,
            # output_dir=f'outputs_{out_dir}'
        ),
        data_collator=DataCollatorForLanguageModeling(tokenizer,
                                                      mlm=False)
    )

    # Silence the warnings
    model.config.use_cache = False
    trainer.train()

    # Save the model
    model.save_pretrained(output_model_dir)

    return model, tokenizer


def create_test_prompted_text(dataset: pd.DataFrame,
                              label_name: str) -> List[str]:
    """
    Creates prompted text for classification from the test dataset.

    Args:
        dataset (pd.DataFrame): The dataset containing interview questions
        and answers.
        label_name (str): The name of the label column for classification.

    Returns:
        List[str]: A list of formatted prompt texts for each interview response.
    """

    texts = []
    classes_names = ', '.join(list(dataset[label_name].unique()))

    for _, row in dataset.iterrows():
        texts.append(
            f"You will be given a part of an interview."
            f"Classify the response to the selected question"
            f"into one of the following categories: {classes_names}"
            f". \n\n ### Part of the interview ### \nIntervier:"
            f" {row['interview_question']} \nResponse:"
            f" {row['interview_answer']} \n\n### Selected Question ###\n"
            f"{row['question']} \n\nLabel:"
        )
    return texts


def create_inference_prompted_text(dataset: pd.DataFrame,
                                   label_name: str) -> List[str]:
    """
    Creates prompted text for classification from the test dataset.

    Args:
        dataset (pd.DataFrame): The dataset containing interview questions
        and answers.
        label_name (str): The name of the label column for classification.

    Returns:
        List[str]: A list of formatted prompt texts for each interview response.
    """

    texts = []
    classes_names = ', '.join(list(dataset[label_name].unique()))

    for _, row in dataset.iterrows():
        texts.append(
            f"You will be given a part of an interview."
            f"Classify the response to the selected question"
            f"into one of the following categories: {classes_names}"
            f". \n\n ### Part of the interview ### \nIntervier:"
            f" {row['interview_question']} \nResponse:"
            f" {row['interview_answer']} \n\n### Selected Question ###\n"
            f"{row['question']} \n\nLabel:"
        )
    return texts


def predict(test: pd.DataFrame,
            categories: list,
            model: nn.Module,
            tokenizer: PreTrainedTokenizer) -> list:
    """
    Generates predictions for the test dataset using the provided model
    and tokenizer.

    Args:
        test (pd.DataFrame): The test dataset containing prompts.
        categories (list): The list of possible categories for classification.
        model (nn.Module): The trained model for making predictions.
        tokenizer (PreTrainedTokenizer): The tokenizer associated with the model.

    Returns:
        list: A list of predicted labels for the test dataset.
    """
    batch_size = 8
    category_set = set(category.lower() for category in categories)
    model.eval()  # Set the model to evaluation mode
    model.config.use_cache = False

    with torch.no_grad(), torch.autocast("cuda"):
        y_pred = []

        pipe = pipeline(task="text-generation",
                        model=model,
                        tokenizer=tokenizer)
        # NEW
        terminators = [
            pipe.tokenizer.eos_token_id,
            pipe.tokenizer.convert_tokens_to_ids("<|eot_id|>")
        ]

        for i in tqdm(range(0, len(test), batch_size)):
            prompts = test.iloc[i:i + batch_size]["text"].tolist()
            results = pipe(prompts,
                           eos_token_id=terminators)

            for result in results:
                answer = result[0]['generated_text'].split("Label:")[-1].strip()
                matched = False

                for category in category_set:
                    if category in answer.lower():
                        print(f"Right label: {answer.lower()}")
                        y_pred.append(category)
                        matched = True
                        break

                if not matched:
                    print(f"Wrong label: {answer.lower()}")
                    y_pred.append("none")

    return y_pred


def evaluation_report(y_true: pd.Series,
                      y_pred: pd.Series,
                      labels: list,
                      run=None) -> None:
    """
    Generates and prints an evaluation report including accuracy and 
    classification metrics.
    
    Args:
        y_true (np.ndarray): The true labels for the test dataset.
        y_pred (np.ndarray): The predicted labels for the test dataset.
        labels (list): The list of label names.
        run: Optional; a wandb run object for logging metrics.
    """
    mapping = {label: idx for idx, label in enumerate(labels)}

    def map_func(x):
        return mapping.get(x, -1)  # Map to -1 if not found

    y_true_mapped = np.vectorize(map_func)(y_true)
    y_pred_mapped = np.vectorize(map_func)(y_pred)

    if run:
        wandb_log_dict = {}

    # Calculate accuracy
    accuracy = accuracy_score(y_true=y_true_mapped, y_pred=y_pred_mapped)
    print(f'Accuracy: {accuracy:.2f}')
    if run:
        wandb_log_dict["Accuracy"] = accuracy

    # Generate accuracy report
    unique_labels = set(y_true_mapped)  # Get unique labels

    for label in unique_labels:
        label_indices = [i for i in range(len(y_true_mapped))
                         if y_true_mapped[i] == label]
        label_y_true = [y_true_mapped[i] for i in label_indices]
        label_y_pred = [y_pred_mapped[i] for i in label_indices]
        label_accuracy = accuracy_score(label_y_true, label_y_pred)
        print(f'Accuracy for label {labels[label]}: {label_accuracy:.2f}')
        if run:
            wandb_log_dict[f"Accuracy for label {labels[label]}"] = label_accuracy

    unsplit_labels = [label.replace(" ", "_") for label in labels]

    # Generate classification report
    class_report = classification_report(y_true=y_true_mapped,
                                         y_pred=y_pred_mapped,
                                         target_names=unsplit_labels,
                                         labels=list(range(len(labels))))
    print('\nClassification Report:')
    print(class_report)

    report_columns = ["Class", "Precision", "Recall", "F1-score", "Support"]
    report_table = []
    class_report = class_report.splitlines()
    for line in class_report[2:(len(labels)+2)]:
        report_table.append(line.split())

    if run:
        wandb_log_dict["Classification Report"] = wandb.Table(
            data=report_table,
            columns=report_columns)

    # For not predicted classes
    mask = y_pred_mapped != -1
    y_true_mapped2 = y_true_mapped[mask]
    y_pred_mapped2 = y_pred_mapped[mask]

    # Generate confusion matrix
    conf_matrix = confusion_matrix(y_true=y_true_mapped,
                                   y_pred=y_pred_mapped,
                                   labels=list(range(len(labels))))
    print('\nConfusion Matrix:')
    print(conf_matrix)

    if run:
        wandb_log_dict["Confusion Matix"] = wandb.plot.confusion_matrix(
            y_true=y_true_mapped2,
            preds=y_pred_mapped2,
            class_names=labels
        )
        run.log(wandb_log_dict)


# !TODO: Load model from file
def evaluate(base_model_name: str,
             fine_tuned_model_path: str,
             label_name: str,
             model: nn.Module = None,
             tokenizer: PreTrainedTokenizer = None,
             run=None) -> None:
    """
    Evaluates the fine-tuned model on the test dataset and
    generates an evaluation report.

    Args:
        base_model_name (str): The name of the base model to load.
        fine_tuned_model_path (str): The path to the fine-tuned model.
        label_name (str): The name of the label column for classification.
        model (nn.Module): Optional; a pre-trained model to use for evaluation.
        tokenizer (PreTrainedTokenizer): Optional; a tokenizer to use for evaluation.
        run: Optional; a wandb run object for logging metrics.
    """

    if not model:

        model = AutoModelForCausalLM.from_pretrained(
            base_model_name,
            return_dict=True,
            low_cpu_mem_usage=True,
            quantization_config=BitsAndBytesConfig(
                load_in_4bit=True,
                bnb_4bit_compute_dtype=torch.bfloat16
            ),
            # torch_dtype=torch.float16,
            device_map="auto",
            trust_remote_code=True,
            offload_folder="offload/",
            cache_dir=cache_dir
        )
    if not tokenizer:
        tokenizer = AutoTokenizer.from_pretrained(base_model_name,
                                                  cache_dir=cache_dir)

    tokenizer.add_special_tokens({'pad_token': '[PAD]'})

    # Get test set data
    test_df = pd.read_csv('preprocessed_data/test_set.csv')[[
        'question',
        'interview_question',
        'interview_answer',
        label_name
    ]][:20]

    test_texts = create_test_prompted_text(test_df, label_name)
    dataset = pd.DataFrame(test_texts, columns=['text'])

    labels = [label.lower() for label in list(test_df[label_name].unique())]

    y_pred = predict(dataset, labels, model, tokenizer)
    y_pred = pd.Series(y_pred, name=label_name)
    y_true = test_df[label_name].str.lower()
    evaluation_report(y_true, y_pred, labels, run)


# !TODO: Load model from file
def inference(base_model_name: str,
              fine_tuned_model_path: str,
              label_name: str,
              model: nn.Module = None,
              tokenizer: PreTrainedTokenizer = None,
              ) -> None:
    """
    Inference a model on the test dataset and
    generates an evaluation report.

    Args:
        base_model_name (str): The name of the base model to load.
        fine_tuned_model_path (str): The path to the fine-tuned model.
        label_name (str): The name of the label column for classification.
        model (nn.Module): Optional; a pre-trained model to use for inference.
        tokenizer (PreTrainedTokenizer): Optional; a tokenizer to use for inference.
    """

    if not model:

        model = AutoModelForCausalLM.from_pretrained(
            base_model_name,
            return_dict=True,
            low_cpu_mem_usage=True,
            quantization_config=BitsAndBytesConfig(
                load_in_4bit=True,
                bnb_4bit_compute_dtype=torch.bfloat16
            ),
            # torch_dtype=torch.float16,
            device_map="auto",
            trust_remote_code=True,
            offload_folder="offload/",
            cache_dir=cache_dir
        )
    if not tokenizer:
        tokenizer = AutoTokenizer.from_pretrained(base_model_name,
                                                  cache_dir=cache_dir)

    tokenizer.add_special_tokens({'pad_token': '[PAD]'})

    # Get test set data
    test_df = pd.read_csv('preprocessed_data/test_set.csv')[[
        'question',
        'interview_question',
        'interview_answer',
        label_name
    ]]  # [:20]

    test_texts = create_inference_prompted_text(test_df, label_name)
    dataset = pd.DataFrame(test_texts, columns=['text'])

    labels = [label.lower() for label in list(test_df[label_name].unique())]

    y_pred = predict(dataset, labels, model, tokenizer)
    y_pred = pd.Series(y_pred, name=label_name)
    y_true = test_df[label_name].str.lower()
    evaluation_report(y_true, y_pred, labels)

# Model fine-tuning and evaluation                                   

In [ ]:
os.environ['WANDB_NOTEBOOK_NAME'] = 'lora_llm'
os.environ["WANDB_DISABLED"] = "false"

# Load the API key from the secret.json file
with open('secrets.json', 'r') as file:
    secrets = json.load(file)
    huggingface_hub.login(secrets.get('HF_KEY'))
    wandb.login(key=secrets.get('WANDB_KEY'))

base_model_name = 'meta-llama/Llama-3.1-8B-Instruct'
fine_tuned_model_path = "./llama3.1"
label_name = "evasion_label"
lr = 2e-4
epochs = 1
class_names = []

# Wandb configuration
# run = wandb.init(entity="kontilenia-national-technical-university-of-athens",
#                  project='political-speech-clarity',
#                  job_type="training",
#                  name=str(epochs)+" epoch Llama3.1 8b - TO DELETE",
#                  # Track hyperparameters and run metadata
#                  config={
#                     "learning_rate": lr,
#                     "architecture": base_model_name,
#                     "dataset": "qevasion_dataset_preproccessed",
#                     "epochs": epochs,
#                  })

run = wandb.init(entity="kontilenia-national-technical-university-of-athens",
                 project="political-speech-clarity",
                 id="dnby3t8o",
                 resume="must")

model, tokenizer = finetuning(base_model_name,
                              fine_tuned_model_path,
                              label_name,
                              lr,
                              epochs)


evaluate(base_model_name,
         fine_tuned_model_path,
         "clarity_label",
         model,
         tokenizer,
         run)

run.finish()

# Prompt tuning and model evaluation                          

In [22]:
# Load the API key from the secret.json file
with open('secrets.json', 'r') as file:
    secrets = json.load(file)
    huggingface_hub.login(secrets.get('HF_KEY'))

base_model_name = "meta-llama/Llama-3.1-70B-Instruct"

model = AutoModelForCausalLM.from_pretrained(
    base_model_name,
    device_map="auto",
    torch_dtype="float16",
    cache_dir=cache_dir,
    quantization_config=BitsAndBytesConfig(
            load_in_4bit=True,
            bnb_4bit_compute_dtype=torch.bfloat16
        ),
)

model.config.use_cache = False
model.config.pretraining_tp = 1

tokenizer = AutoTokenizer.from_pretrained(base_model_name, cache_dir=cache_dir)

tokenizer.pad_token_id = tokenizer.eos_token_id

inference(base_model_name,
          "",
          "clarity_label",
          model,
          tokenizer)

Device set to use cuda:0
  3%|▎         | 1/39 [01:50<1:09:40, 110.02s/it]

Right label: indirect 
reason: the response did not directly answer the question about the redline, but instead diverted the topic to a broader discussion about the international response to the north korean regime's actions. 

### explanation of labels: 
- indirect: the response does not directly answer the question, often diverting the topic or providing a vague statement. 
- direct reply: the response clearly and concisely answers the question, providing the requested information. 
- direct non-reply: the response explicitly states that the question will not be answered or provides a clear refusal to comment.
Right label: direct reply 

explanation: the response provides a clear and direct answer to the question, stating the conditions under which they would invite others to the white house to negotiate on the jobs bill.
Right label: direct non-reply

reason: the response did not directly address the question about the necessity of japan dropping the threat of sanctions from its pro

  5%|▌         | 2/39 [03:11<57:40, 93.52s/it]   

Right label: direct non-reply
reason: the question asks what the speaker is offering arab nations in order to encourage them to participate. however, the speaker's response does not address the issue of what is being offered. instead, the speaker talks about the importance of arab buy-in for a state and the need for the arab nations to be part of the process. the speaker does not provide a direct answer to the question.
Right label: direct non-reply 

explanation: the response does not address the question directly. the president avoids providing a straightforward answer to whether tehran's influence is growing despite us efforts, instead focusing on the broader threat of terrorism, the importance of international cooperation, and the need to advance liberty and defeat an ideology that doesn't believe in freedom. the response includes some relevant information, but it does not provide a clear answer to the question.
Right label: indirect 

reason: the response did not answer the questi

  8%|▊         | 3/39 [05:21<1:06:01, 110.04s/it]

...

Right label: direct non-reply

the response does not answer the question about president kikwete's thoughts on america electing a black president with african roots. instead, president kikwete deflects the question and focuses on the upcoming change in the u.s. presidency, expressing their hope for a good friend of africa regardless of who becomes the new president.
Right label: **direct reply** 
reason: the response directly answers the question and provides a clear explanation for the u.s. boycotting the conference. the president expresses his concerns about the conference, specifically the language adopted from a previous conference in 2001, and explains that the u.s. would be happy to participate if they had a "clean start" but cannot participate if the objectionable provisions are incorporated.
Right label: indirect reply 

### rationale ###
the response does not directly address the question of what is different about the current set of warnings. instead, it focuses on the import

 97%|█████████▋| 38/39 [1:19:23<01:52, 112.74s/it]

Right label: direct reply

### reasoning skill for direct reply ###
the respondent directly answers the question by stating that he will work to build bridges and promote interfaith harmony, as evidenced by his statement "i'll work hard to do my part." he also mentions specific actions he has taken and will continue to take, such as articulating a two-state solution for the israeli-palestinian conflict and providing aid to those in need. the respondent provides a clear and direct response to the question, indicating his willingness to take a leading role in promoting interfaith harmony.
Right label: direct non-reply

the response does not address the question about the beirut airport bombing. instead, it changes the subject to an unrelated topic (the pig). therefore, it is a direct non-reply.
Right label: direct reply 
reason: the response is a direct answer to the question. the respondent expresses his thoughts on the issue.
Right label: indirect. the response doesn't answer the quest

100%|██████████| 39/39 [1:20:42<00:00, 124.16s/it]

Right label: indirect reply 
justification: the speaker's response does not address the requested topic of the frustration of americans, instead it discusses a range of other topics including the 1559 resolution, the role of the united states in the middle east, the importance of liberty, and the threat of islamic fascism. the speaker's response is not relevant to the question asked and does not provide a clear answer to the question. 

please let me know if you want me to classify another response.
Right label: direct non-reply.
Right label: direct reply 

reasoning skill: this response is classified as a direct reply because it provides a clear and specific answer to the question. the speaker acknowledges that they had initial concerns about the amount of funding congress would allocate, but expresses satisfaction with the $110 billion commitment. they also discuss specific examples of bureaucratic obstacles they've encountered and how they're working to address them. the response is

In [23]:
!df -h

Filesystem      Size  Used Avail Use% Mounted on
devtmpfs        374G     0  374G   0% /dev
tmpfs           374G   44K  374G   1% /dev/shm
tmpfs           374G  1.5M  374G   1% /run
tmpfs           374G     0  374G   0% /sys/fs/cgroup
/dev/nvme0n1p1  135G   82G   54G  61% /
tmpfs            75G     0   75G   0% /run/user/0
/dev/nvme3n1    2.0T  142G  1.8T   8% /home/ec2-user/SageMaker
tmpfs            75G     0   75G   0% /run/user/1000
tmpfs            75G     0   75G   0% /run/user/1002
tmpfs            75G     0   75G   0% /run/user/1001
